In [1]:
import os
import pandas as pd
root = './data/celeba'
attributes = pd.read_csv(os.path.join(root, 'list_attr_celeba.csv'))
images_dir = os.path.join(root, 'img_align_celeba')


In [2]:
from src.models.autoencoder import AutoEncoder

In [3]:
good_attribute_threshold = 0.3
good_attributes = ['image_id']
for key in attributes.keys():
    c = attributes[attributes[key]==1]['image_id'].count()
    if c < len(attributes)*(1-good_attribute_threshold) and c > len(attributes) * good_attribute_threshold:
        good_attributes.append(key)
gdf = attributes[good_attributes]

In [4]:
list(attributes.keys()).index("Mouth_Slightly_Open")
from argparse import Namespace
print(good_attributes)
print(list(attributes.keys()).index("Attractive"))

['image_id', 'Attractive', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Smiling', 'Wavy_Hair', 'Wearing_Lipstick']
3


In [5]:
args = Namespace(
    log_every_n_steps=1,
    gradient_clip_val=0.5,
    learning_rate=1e-3,
    weight_decay=1e-6,
    precision=16,
    data_dir = './data/celeba',
    batch_size = 512,
    num_workers = 4,
    gpus=2,
    max_epochs=10,
    accumulate_grad_batches=4,
)

In [6]:
from argparse import ArgumentParser
from src.models.classifier import Classifier
from src.datamodule import CelebAData
from src.models.cnn_for_encoded import SimpleCNN
import pytorch_lightning as pl
cnn = SimpleCNN().to('cuda:0')
from torchvision.models import resnet34
for i in range(len(attributes.keys())-1):
    target_attr = i
    pre = AutoEncoder.load_from_checkpoint(hparams=args, checkpoint_path='logs/autoencoder/0114/200epochs/adam/version_6/checkpoints/epoch=187-step=15039.ckpt').encoder.eval()
    classifier = Classifier(args, model=cnn, target_attr=target_attr, preEncoder=pre)
    from pytorch_lightning.loggers import TensorBoardLogger
    logger = TensorBoardLogger("logs/cls/{}".format(attributes.keys()[1:][i]), name='')
    import sys
    sys.argv = ['-f']
    parser = ArgumentParser()
    parser = pl.Trainer.add_argparse_args(parser)
    argu = vars(parser.parse_args())
    argu.update(vars(args))
    args = Namespace(**argu)
    dm = CelebAData(args)
    trainer = pl.Trainer.from_argparse_args(args, logger=logger)
    trainer.fit(classifier, datamodule=dm)

/home/hjsong/anaconda3/envs/ppml/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/hjsong/anaconda3/envs/ppml/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:849: UserWarning: You requested multiple GPUs but did not specify a backend, e.g. `Trainer(strategy="dp"|"ddp"|"ddp2")`. Setting `strategy="ddp_spawn"` for you.
  rank_zero_warn(
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backe

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

ProcessRaisedException: 

-- Process 0 terminated with the following error:
Traceback (most recent call last):
  File "/home/hjsong/anaconda3/envs/ppml/lib/python3.8/site-packages/torch/multiprocessing/spawn.py", line 59, in _wrap
    fn(i, *args)
  File "/home/hjsong/anaconda3/envs/ppml/lib/python3.8/site-packages/pytorch_lightning/plugins/training_type/ddp_spawn.py", line 208, in _wrapped_function
    result = function(*args, **kwargs)
  File "/home/hjsong/anaconda3/envs/ppml/lib/python3.8/site-packages/pytorch_lightning/plugins/training_type/ddp_spawn.py", line 236, in new_process
    results = trainer.run_stage()
  File "/home/hjsong/anaconda3/envs/ppml/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 1284, in run_stage
    return self._run_train()
  File "/home/hjsong/anaconda3/envs/ppml/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 1306, in _run_train
    self._run_sanity_check(self.lightning_module)
  File "/home/hjsong/anaconda3/